# 🔍 Elliptic Senior Data Scientist Takehome – Crypto Wallet Investigation

In this notebook, I support the investigations team by identifying a suspect wallet on the Tron blockchain using behavioral and transactional patterns. Based on the intelligence provided, I implement a structured filtering pipeline to isolate the criminal wallet.

## 1. Load and Preview the Dataset

I load a Parquet file containing Tron-based token transfers. Each row corresponds to a transaction with its timestamp, value (in USD and token units), source and destination addresses, and token type.

In [102]:
import pandas as pd

# Load dataset
df = pd.read_parquet("data/tron_blockchain_interview_data.parquet")

# Convert timestamp
df['datetime'] = pd.to_datetime(df['unixtimestamp'], unit='s')

# Preview data
df.head()### 3. Filter and Identify the Suspect Wallet

To identify the suspect wallet, I applied a structured multi-step filter based on the behavioral intelligence provided:

- I began by filtering all **USDT transactions in January 2025**.
- I then isolated wallets that made **exactly 3 transactions** during that period.
- For those wallets, I checked for transactions on **January 3** that **summed to approximately \$800**, grouping by sender and receiver to consider daily totals.
- From those candidates, I verified if they also made **a single transaction around \$25 before January 3**.
- Finally, I ensured the **\$25 transaction occurred only once per wallet** to match the behavior exactly.

This process helped narrow down the list to wallets that closely align with the fraudulent behavior described in the intelligence report.

,unixtimestamp,transaction,fromAddress,toAddress,token,token_amount,usd_value,datetime
0,1735742709,8a8b7e3c51f1e15e8fb77d38e5f44fcc319a2053bee4d5...,TJ7hhYhVhaxNx6BPyq7yFpqZrQULL3JSdb,TPq6UAn94sKqVmtxybdMemVWiDcpQhAx43,USDT,846000000,844.310401,2025-01-01 14:45:09
1,1735742709,757cdf71a34741a5a6e1c085dd2408e4f6f49a67a77a6b...,TEmb2qSrWtmsx4F8aijGAy3CRApKLx3JtT,TCZ4tPauSPj6bAoYbJYq8TQvaYziXiZqLS,USDT,279000000,278.442792,2025-01-01 14:45:09
2,1735742709,12aa49b2915a33f37da62b1c04451cc96c3aeea7bbbc1c...,TMQkP3upAn4reUUw5HeXVWy6PjhWw3kVbz,TWAudzu86TJ8DtHb4wBvxWx2EpmqtcD3Wb,TRX,4300000,1.089288,2025-01-01 14:45:09
3,1735742709,faa2668b2864f2934b210c30bb1dd18d01c5eb95721732...,TLj5jNLzaoR94c5WUH9uxpQAY3RZT6gh2y,TWg8JWoB646F6DRAU8vkKMUsSiKa4htV2v,USDT,100166300,99.966252,2025-01-01 14:45:09
4,1735742709,8dcbb9d945567e210ff8d86e8157c5fc75ca0a557d6568...,TL6h4ANWARrjTnSK1HYNfDc9m5HV9GDKoZ,TTCtwXVkHwmcEQ9od7aW8WMp72dmzxMFhB,USDT,52460000,52.355229,2025-01-01 14:45:09


### 2. Filter and Identify the Suspect Wallet

To identify the suspect wallet, I applied a structured multi-step filter based on the behavioral intelligence provided:

- I began by filtering all **USDT transactions in January 2025**.
- I then isolated wallets that made **exactly 3 transactions** during that period.
- For those wallets, I checked for transactions on **January 3** that **summed to approximately \$800**, grouping by sender and receiver to consider daily totals.
- From those candidates, I verified if they also made **a single transaction around \$25 before January 3**.
- Finally, I ensured the **\$25 transaction occurred only once per wallet** to match the behavior exactly.

This process helped narrow down the list to wallets that closely align with the fraudulent behavior described in the intelligence report.

In [100]:
from datetime import date
import pandas as pd

# ## 1. Load USDT transactions for January 2025
jan_2025_usdt = df[
    (df['datetime'] >= '2025-01-01') &
    (df['datetime'] < '2025-02-01') &
    (df['token'] == 'USDT')
].copy()

# ## 2. Filter wallets with exactly 3 transactions
tx_count = jan_2025_usdt['fromAddress'].value_counts()
wallets_with_3_txns = tx_count[tx_count == 3].index
three_txns_df = jan_2025_usdt[jan_2025_usdt['fromAddress'].isin(wallets_with_3_txns)].copy()
three_txns_df['date'] = three_txns_df['datetime'].dt.date

# ## 3. Find $800 transactions on Jan 3 (grouped daily totals)
jan_3_txns = three_txns_df[three_txns_df['date'] == date(2025, 1, 3)]
grouped_jan_3 = jan_3_txns.groupby(['fromAddress', 'toAddress', 'date'], as_index=False)['usd_value'].sum()
jan_3_grouped_800 = grouped_jan_3[grouped_jan_3['usd_value'].between(799, 801)]

# ## 4. Get candidate wallets with ~800 USD total on Jan 3
wallets_800_jan3 = grouped_jan_3[grouped_jan_3['usd_value'].between(795, 805)]['fromAddress'].unique()

# ## 5. Check for ~$25 transaction before Jan 3
candidate_txns = three_txns_df[
    (three_txns_df['fromAddress'].isin(wallets_800_jan3)) &
    (three_txns_df['date'] < date(2025, 1, 3)) &
    (three_txns_df['usd_value'].between(24, 26))
]

# ## 6. Count how many wallets satisfy the ~$25 condition
condition_25_summary = candidate_txns['fromAddress'].value_counts()
condition_25_summary


fromAddress
TKdgKMPXtXwwQi5bVzutBisbsMSFSvP9xm    1
Name: count, dtype: int64

In [101]:
jan_2025_usdt[jan_2025_usdt['fromAddress'] == 'TKdgKMPXtXwwQi5bVzutBisbsMSFSvP9xm']

,unixtimestamp,transaction,fromAddress,toAddress,token,token_amount,usd_value,datetime
39908,1735802904,a129572d10dc73c06842c1c49f0177c3f16c07b9bac372...,TKdgKMPXtXwwQi5bVzutBisbsMSFSvP9xm,TY9a92BhjcNB7UzYihdh7wDxCrtKYuDzsf,USDT,25590000,25.545366,2025-01-02 07:28:24
229495,1736520390,83ad75043d490a30261c6b6d7ac4c74543aa95002337af...,TKdgKMPXtXwwQi5bVzutBisbsMSFSvP9xm,TKf1C66x42nzHj9t3LZceckMUUobJ5VcjF,USDT,690000000,689.952753,2025-01-10 14:46:30
623203,1735874895,38430bd0dd49502093b5530dfed48b4d7ae559db7a49f5...,TKdgKMPXtXwwQi5bVzutBisbsMSFSvP9xm,TKf1C66x42nzHj9t3LZceckMUUobJ5VcjF,USDT,800000000,799.391280,2025-01-03 03:28:15


In [98]:
jan_2025_usdt[jan_2025_usdt['fromAddress'] == 'TKdgKMPXtXwwQi5bVzutBisbsMSFSvP9xm']

,unixtimestamp,transaction,fromAddress,toAddress,token,token_amount,usd_value,datetime
39908,1735802904,a129572d10dc73c06842c1c49f0177c3f16c07b9bac372...,TKdgKMPXtXwwQi5bVzutBisbsMSFSvP9xm,TY9a92BhjcNB7UzYihdh7wDxCrtKYuDzsf,USDT,25590000,25.545366,2025-01-02 07:28:24
229495,1736520390,83ad75043d490a30261c6b6d7ac4c74543aa95002337af...,TKdgKMPXtXwwQi5bVzutBisbsMSFSvP9xm,TKf1C66x42nzHj9t3LZceckMUUobJ5VcjF,USDT,690000000,689.952753,2025-01-10 14:46:30
623203,1735874895,38430bd0dd49502093b5530dfed48b4d7ae559db7a49f5...,TKdgKMPXtXwwQi5bVzutBisbsMSFSvP9xm,TKf1C66x42nzHj9t3LZceckMUUobJ5VcjF,USDT,800000000,799.391280,2025-01-03 03:28:15


### 4. Final Result

The wallet that matched **all behavioral criteria** is:

#### `TKdgKMPXtXwwQi5bVzutBisbsMSFSvP9xm`

This wallet:

- Made exactly **3 USDT transactions** in **January 2025**  
- Sent approximately **USD 800 on January 3** (daily total)  
- Sent **USD 25 before January 3**  
- Sent between **USD 500–USD 700 on January 10**

It is the **only wallet** satisfying all conditions in the intelligence report.